## Imports

In [1]:
import sys
import os
import random
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
import h5py
from h5py import File as HDF5File

import enum
print(enum.__file__)

/usr/lib/python3.8/enum.py


In [3]:
# Oh boy this might take a while
import tensorflow as tf
from tensorflow import keras

from keras.layers import Lambda, Input
from keras.layers import Dropout, Flatten, Dense
import keras.backend as K
from keras.models import Sequential, Model 
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import concatenate

2024-01-16 04:25:44.701032: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-16 04:25:45.513503: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import yaml
with open("../config.yaml") as fin:
    config = yaml.safe_load(fin)
    data_dir = config["data_dir"]
    model_dir = config["model_dir"]
    output_dir = config["output_dir"]

In [5]:
task_name = "scalar1"

## Loading data

See `cnn/data.py`.

In [6]:
from cnn_data import load_data
X_train, Y_train, X_test, Y_test = load_data(task_name)

load_data from datafile /data/wifeng/photon-jet/data/h5/pi0_40-250GeV_100k.h5
load_data from datafile /data/wifeng/photon-jet/data/h5/gamma_40-250GeV_100k.h5
load_data from datafile /data/wifeng/photon-jet/data/h5/scalar1_40-250GeV_100k.h5


In [7]:
print("Training shapes:")
print("  inputs:", X_train[0].shape, X_train[1].shape)
print("  labels:", Y_train.shape)

Training shapes:
  inputs: (21000, 4, 16, 1) (21000, 4, 128, 1)
  labels: (21000,)


## Build the model

In [8]:
visible1 = Input(shape=(4, 16, 1))
conv11 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible1)
pool11 = MaxPooling2D(pool_size=(2, 2))(conv11)
conv12 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool11)
pool12 = MaxPooling2D(pool_size=(2, 2))(conv12)
flat1 = Flatten()(pool12)
print ('cov11.shape', conv11.shape)
print ('pool1.shape', pool11.shape)
print ('cov12.shape', conv12.shape)
print ('pool2.shape', pool12.shape)

cov11.shape (None, 4, 16, 32)
pool1.shape (None, 2, 8, 32)
cov12.shape (None, 2, 8, 16)
pool2.shape (None, 1, 4, 16)


2024-01-16 04:25:53.930280: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9804 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:89:00.0, compute capability: 7.5
2024-01-16 04:25:53.931398: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9804 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:b1:00.0, compute capability: 7.5
2024-01-16 04:25:53.932291: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 9804 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:b2:00.0, compute capability: 7.5


In [9]:
visible2 = Input(shape=(4, 128, 1))
conv21 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible2)
pool21 = MaxPooling2D(pool_size=(2, 2))(conv21)
conv22 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool21)
pool22 = MaxPooling2D(pool_size=(2, 2))(conv22)
flat2 = Flatten()(pool22)
print ('cov21.shape', conv21.shape)
print ('poo21.shape', pool21.shape)
print ('cov22.shape', conv22.shape)
print ('poo22.shape', pool22.shape)

cov21.shape (None, 4, 128, 32)
poo21.shape (None, 2, 64, 32)
cov22.shape (None, 2, 64, 16)
poo22.shape (None, 1, 32, 16)


In [10]:
visible3 = Input(shape=(16, 16, 1))
conv31 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible3)
pool31 = MaxPooling2D(pool_size=(2, 2))(conv31)
conv32 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool31)
pool32 = MaxPooling2D(pool_size=(2, 2))(conv32)
flat3 = Flatten()(pool32)
print ('cov31.shape', conv31.shape)
print ('poo31.shape', pool31.shape)
print ('cov32.shape', conv32.shape)
print ('poo32.shape', pool32.shape)

cov31.shape (None, 16, 16, 32)
poo31.shape (None, 8, 8, 32)
cov32.shape (None, 8, 8, 16)
poo32.shape (None, 4, 4, 16)


In [11]:
visible4 = Input(shape=(16, 8, 1))
conv41 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible4)
pool41 = MaxPooling2D(pool_size=(2, 2))(conv41)
conv42 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool41)
pool42 = MaxPooling2D(pool_size=(2, 2))(conv42)
flat4 = Flatten()(pool32)
print ('cov41.shape', conv41.shape)
print ('poo41.shape', pool41.shape)
print ('cov42.shape', conv42.shape)
print ('poo42.shape', pool42.shape)

cov41.shape (None, 16, 8, 32)
poo41.shape (None, 8, 4, 32)
cov42.shape (None, 8, 4, 16)
poo42.shape (None, 4, 2, 16)


In [12]:
merge = concatenate([flat1, flat2, flat3, flat4])

In [13]:
# interpretation model
hidden1 = Dense(20, activation='relu')(merge)
hidden2 = Dense(20, activation='relu')(hidden1)
output = Dense(3)(hidden2)
cnn = Model(inputs=[visible1, visible2, visible3, visible4], outputs=output)

In [14]:
# summarize layers
# print(cnn.summary())

In [15]:
cnn.inputs

[<KerasTensor: shape=(None, 4, 16, 1) dtype=float32 (created by layer 'input_1')>,
 <KerasTensor: shape=(None, 4, 128, 1) dtype=float32 (created by layer 'input_2')>,
 <KerasTensor: shape=(None, 16, 16, 1) dtype=float32 (created by layer 'input_3')>,
 <KerasTensor: shape=(None, 16, 8, 1) dtype=float32 (created by layer 'input_4')>]

## Train the model

In [16]:
def train_iteration(lr, epochs):
    print(f"=== Training with lr={lr} for {epochs} epochs ===")
    cnn.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"]
    )
    return cnn.fit(
        X_train, Y_train,
        epochs=epochs, batch_size=128,
        validation_data=(X_test, Y_test),
    )

In [17]:
train_iteration(lr=1e-2, epochs=5)

=== Training with lr=0.01 for 5 epochs ===
Epoch 1/5


2024-01-16 04:25:56.525428: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8700
2024-01-16 04:25:56.693052: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-16 04:25:57.051744: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fbf8225d840 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-16 04:25:57.051799: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-01-16 04:25:57.051810: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-01-16 04:25:57.051819: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (2): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-01-16 04:25:57.059153: I tensorflow/compiler/mlir/tensorflow/utils/du

165/165 [==============================] - 8s 18ms/step - loss: 68.0389 - accuracy: 0.5476 - val_loss: 0.6013 - val_accuracy: 0.6173
Epoch 2/5
165/165 [==============================] - 1s 9ms/step - loss: 0.5555 - accuracy: 0.7540 - val_loss: 1.2429 - val_accuracy: 0.6164
Epoch 3/5
165/165 [==============================] - 2s 12ms/step - loss: 0.5252 - accuracy: 0.7705 - val_loss: 0.5030 - val_accuracy: 0.7619
Epoch 4/5
165/165 [==============================] - 2s 10ms/step - loss: 0.3583 - accuracy: 0.8797 - val_loss: 0.2021 - val_accuracy: 0.9403
Epoch 5/5
165/165 [==============================] - 2s 9ms/step - loss: 0.1679 - accuracy: 0.9515 - val_loss: 0.1922 - val_accuracy: 0.9453


In [18]:
train_iteration(lr=2e-3, epochs=5)

=== Training with lr=0.002 for 5 epochs ===
Epoch 1/5
165/165 [==============================] - 5s 16ms/step - loss: 0.1323 - accuracy: 0.9619 - val_loss: 0.0771 - val_accuracy: 0.9767
Epoch 2/5
165/165 [==============================] - 2s 10ms/step - loss: 0.0656 - accuracy: 0.9796 - val_loss: 0.1388 - val_accuracy: 0.9506
Epoch 3/5
165/165 [==============================] - 2s 11ms/step - loss: 0.0704 - accuracy: 0.9788 - val_loss: 0.0640 - val_accuracy: 0.9807
Epoch 4/5
165/165 [==============================] - 1s 9ms/step - loss: 0.0561 - accuracy: 0.9828 - val_loss: 0.0765 - val_accuracy: 0.9768
Epoch 5/5
165/165 [==============================] - 1s 7ms/step - loss: 0.0471 - accuracy: 0.9847 - val_loss: 0.0475 - val_accuracy: 0.9863


In [19]:
train_iteration(lr=1e-3, epochs=5)

=== Training with lr=0.001 for 5 epochs ===
Epoch 1/5
165/165 [==============================] - 5s 16ms/step - loss: 0.0448 - accuracy: 0.9864 - val_loss: 0.0434 - val_accuracy: 0.9870
Epoch 2/5
165/165 [==============================] - 2s 11ms/step - loss: 0.0405 - accuracy: 0.9877 - val_loss: 0.0547 - val_accuracy: 0.9845
Epoch 3/5
165/165 [==============================] - 1s 9ms/step - loss: 0.0395 - accuracy: 0.9879 - val_loss: 0.0511 - val_accuracy: 0.9854
Epoch 4/5
165/165 [==============================] - 1s 8ms/step - loss: 0.0388 - accuracy: 0.9875 - val_loss: 0.0476 - val_accuracy: 0.9862
Epoch 5/5
165/165 [==============================] - 2s 9ms/step - loss: 0.0362 - accuracy: 0.9884 - val_loss: 0.1378 - val_accuracy: 0.9522


In [20]:
train_iteration(lr=2e-4, epochs=5)

=== Training with lr=0.0002 for 5 epochs ===
Epoch 1/5
165/165 [==============================] - 5s 13ms/step - loss: 0.0290 - accuracy: 0.9909 - val_loss: 0.0383 - val_accuracy: 0.9890
Epoch 2/5
165/165 [==============================] - 1s 7ms/step - loss: 0.0260 - accuracy: 0.9920 - val_loss: 0.0436 - val_accuracy: 0.9877
Epoch 3/5
165/165 [==============================] - 1s 7ms/step - loss: 0.0259 - accuracy: 0.9916 - val_loss: 0.0374 - val_accuracy: 0.9891
Epoch 4/5
165/165 [==============================] - 1s 8ms/step - loss: 0.0254 - accuracy: 0.9920 - val_loss: 0.0383 - val_accuracy: 0.9891
Epoch 5/5
165/165 [==============================] - 1s 9ms/step - loss: 0.0238 - accuracy: 0.9930 - val_loss: 0.0385 - val_accuracy: 0.9889


In [21]:
train_iteration(lr=2e-5, epochs=30)

=== Training with lr=2e-05 for 30 epochs ===
Epoch 1/30
165/165 [==============================] - 5s 14ms/step - loss: 0.0218 - accuracy: 0.9931 - val_loss: 0.0373 - val_accuracy: 0.9895
Epoch 2/30
165/165 [==============================] - 1s 9ms/step - loss: 0.0217 - accuracy: 0.9932 - val_loss: 0.0376 - val_accuracy: 0.9894
Epoch 3/30
165/165 [==============================] - 1s 7ms/step - loss: 0.0216 - accuracy: 0.9931 - val_loss: 0.0375 - val_accuracy: 0.9894
Epoch 4/30
165/165 [==============================] - 2s 9ms/step - loss: 0.0215 - accuracy: 0.9932 - val_loss: 0.0374 - val_accuracy: 0.9891
Epoch 5/30
165/165 [==============================] - 1s 8ms/step - loss: 0.0214 - accuracy: 0.9934 - val_loss: 0.0374 - val_accuracy: 0.9895
Epoch 6/30
165/165 [==============================] - 1s 9ms/step - loss: 0.0213 - accuracy: 0.9936 - val_loss: 0.0375 - val_accuracy: 0.9893
Epoch 7/30
165/165 [==============================] - 2s 10ms/step - loss: 0.0213 - accuracy: 0.9933 -

In [22]:
train_iteration(lr=1e-5, epochs=5)

=== Training with lr=1e-05 for 5 epochs ===
Epoch 1/5
165/165 [==============================] - 4s 10ms/step - loss: 0.0197 - accuracy: 0.9939 - val_loss: 0.0375 - val_accuracy: 0.9894
Epoch 2/5
165/165 [==============================] - 1s 6ms/step - loss: 0.0197 - accuracy: 0.9939 - val_loss: 0.0376 - val_accuracy: 0.9894
Epoch 3/5
165/165 [==============================] - 1s 6ms/step - loss: 0.0196 - accuracy: 0.9937 - val_loss: 0.0382 - val_accuracy: 0.9894
Epoch 4/5
165/165 [==============================] - 1s 6ms/step - loss: 0.0197 - accuracy: 0.9938 - val_loss: 0.0376 - val_accuracy: 0.9894
Epoch 5/5
165/165 [==============================] - 1s 8ms/step - loss: 0.0196 - accuracy: 0.9937 - val_loss: 0.0377 - val_accuracy: 0.9894


In [23]:
train_iteration(lr=1e-5, epochs=10)

=== Training with lr=1e-05 for 10 epochs ===
Epoch 1/10
165/165 [==============================] - 5s 11ms/step - loss: 0.0196 - accuracy: 0.9938 - val_loss: 0.0378 - val_accuracy: 0.9895
Epoch 2/10
165/165 [==============================] - 1s 7ms/step - loss: 0.0196 - accuracy: 0.9938 - val_loss: 0.0377 - val_accuracy: 0.9894
Epoch 3/10
165/165 [==============================] - 1s 6ms/step - loss: 0.0196 - accuracy: 0.9939 - val_loss: 0.0378 - val_accuracy: 0.9895
Epoch 4/10
165/165 [==============================] - 1s 6ms/step - loss: 0.0195 - accuracy: 0.9938 - val_loss: 0.0376 - val_accuracy: 0.9894
Epoch 5/10
165/165 [==============================] - 1s 6ms/step - loss: 0.0195 - accuracy: 0.9938 - val_loss: 0.0378 - val_accuracy: 0.9895
Epoch 6/10
165/165 [==============================] - 1s 6ms/step - loss: 0.0195 - accuracy: 0.9940 - val_loss: 0.0377 - val_accuracy: 0.9894
Epoch 7/10
165/165 [==============================] - 1s 6ms/step - loss: 0.0194 - accuracy: 0.9938 - 

In [24]:
# Save this model
os.makedirs(f"{model_dir}", exist_ok=True)

print(f"task name: {task_name}")
cnn.save(f"{model_dir}/{task_name}_cnn")

task name: scalar1
INFO:tensorflow:Assets written to: /data/wifeng/photon-jet/cnn_models_v4.0/scalar1_cnn/assets


INFO:tensorflow:Assets written to: /data/wifeng/photon-jet/cnn_models_v4.0/scalar1_cnn/assets
